In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

setup geometry

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "in"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.2))
mesh.Curve(3);
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

BaseWebGuiScene

setup FEM space

In [3]:
# viscosity
nu = 0.001
# Order of spaces
k = 2
# H1 vs VectorH1 -> vector field?!
V = VectorH1(mesh,order=k, dirichlet="top|bot|cyl|in|out")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q]) # X = [V,V,Q] (without VectorH1)

setup bilinear form
velocityfield u and pressurefield p

In [4]:
u,p = X.TrialFunction()
v,q = X.TestFunction()

a = BilinearForm(X)
a += (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a.Assemble()

setup boundary conditions

In [5]:
gfu = GridFunction(X)
# setup flow condition
uinf = 0.001
uin = CoefficientFunction((uinf,0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("in|top|bot|out"))

velocity = CoefficientFunction(gfu.components[0])
scene = Draw(velocity, mesh, "vel")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

solve equation

In [6]:
def solveStokes():
    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(X.FreeDofs())
    gfu.vec.data += inv * res
    scene.Redraw()
solveStokes()

Drag/ "dissipated energy" [should be working]

$J(\Omega ) = \frac{1}{2} \int_\Omega Du : Du dx$ 

In [7]:
solveStokes()
vel = gfu.components[0] #gfu.vec.data
drag = Integrate(0.5*InnerProduct(Grad(vel),Grad(vel)),mesh)
print(drag)

# 1.0997749905576028e-05 - r=0.5
# 0.00043084429187575745 - r=1.5

1.0997749905576043e-05


In [8]:
def calcDrag(gfu):
    vel = gfu.components[0]
    return Integrate(0.5*InnerProduct(Grad(vel),Grad(vel)),mesh)

### get surface area of mesh [done]
(without ball), should stay constant

In [9]:
surf = Integrate(1,mesh)
def calcSurf():
    return Integrate(1,mesh)

In [10]:
def Cost(gfu):
    return calcDrag(gfu) + abs(calcSurf()-surf)

## Adjoint equation try?
maybe we dont even need this??

In [11]:
r, w = X.TnT()

fadjoint = LinearForm(X)
#fadjoint += -1*(Cost(gfu)).Diff(gfu[0],w)

the rest of the adjoint equation example:
```python
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - aAuto.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = aAuto.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update
fadjoint.Assemble()
SolveAdjointEquation()
scene_p.Redraw()
```

### creation of gfset
function that will perturb our mesh

In [12]:
# Test and trial functions for shape derivate -> do we even need this?
VEC = H1(mesh, order=2, dim=2)
PHI, PSI = VEC.TnT()
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
mesh.SetDeformation(gfset)
sceneSet = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

### Shape derivative $\mathrm{d}J(\Omega)(\mathbf{X}) = \int_{\Omega} \mathbf{S}_1:D \mathbf{X} dx$
$S_1 = \biggl(\frac{1}{2}Du : Du - p div(u) \biggr) I_2 + Du^T p - Du^T Du.$ <br>
$\mathrm{u}$ and $p$ are the solutions to the stokesflow problem<br>
we use u = gfu[0], p = gfu[1]

def S1(u,v):
    return (0.5*InnerProduct(grad(u),grad(u))-p*div(u))*Id(2) + grad(u).trans*p-grad(u).trans*grad(u)

gfset = GridFunction(VEC)
gfset.Set((0,0))

def Equation(u,w,gfset):
    return InnerProduct(S1(u,w),grad(gfset)),mesh

dJOmega = LinearForm(VEC)
#dJOmega += Equation(gfu.components[0], gfu.components[1], gfset)

## symbolicLFI try

In [13]:
dJOmega = LinearForm(VEC)
S1 = (0.5*InnerProduct(grad(u),grad(u))-p*div(u))*Id(2) + grad(u).trans*p-grad(u).trans*grad(u)
shapedif = InnerProduct(S1,grad(gfset))
dJOmega += SymbolicLFI(shapedif)
#dJOmega += SymbolicLFI(Cost(gfu))

In [14]:
b = BilinearForm(VEC)
b += InnerProduct(grad(PSI),grad(PHI))*dx + InnerProduct(PSI,PHI)*dx

gfX = GridFunction(VEC)

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

## this basically just copy paste, not rly knowing what is going on here

In [15]:
b.Assemble()
#dJOmega.Assemble()
#SolveDeformationEquation()
#Draw(-gfX, mesh, "gfX")

In [17]:
# Cost is  calculated for node individually -> change calc

In [18]:
print('Cost at initial design', Integrate (Cost(gfu), mesh))
gfset.Set((0,0))
scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec

Cost at initial design 0.0002553082847066506


ZeroDivisionError: float division by zero

In [ ]:
scene = Draw(gfset)
mesh.SetDeformation(gfset)
scene.Redraw()

a.Assemble()
fstate.Assemble()
SolveStateEquation()
print('Cost at new design', Integrate (Cost(gfu), mesh))